### This notebook is for Sina to run the 3D segmentation and extract the centriod of the neurons

- install tensorflow and stardist

In [1]:
import tensorflow as tf
from stardist import __version__ as stardist_version
print("TensorFlow version:", tf.__version__)
print("StarDist version:", stardist_version)

2025-01-28 18:21:52.490958: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-28 18:22:01.326770: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /shared/centos7/cuda/11.8/lib64:/shared/centos7/nodejs/14.15.4/lib
2025-01-28 18:22:01.327427: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /shared/centos7/cuda/11.8/lib64:/shared/centos7/nodejs/14.15.4/

TensorFlow version: 2.11.0
StarDist version: 0.9.1


- load the modules

In [2]:
import sys
import os
from func_seg import *

True
Tesla V100-PCIE-32GB
True
2.11.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


- load stardist 3D and model weights

In [3]:
model_weights_path = 'weights_best_42stacks_all.h5'
model, model_2D = load_model_3D_and_2D(model_weights_path)

use_gpu:  True


base_model.py (203): output path for model already exists, files may be overwritten: /work/venkatachalamlab/Hang/00Neuron_tracking_version2/01version/Matching_anchor/Sina_seg/models/stardist
2025-01-28 18:22:46.424356: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-28 18:22:46.785624: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31141 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:81:00.0, compute capability: 7.0


Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.707933, nms_thresh=0.3.
There are 4 registered models for 'StarDist2D':

Name                  Alias(es)
────                  ─────────
'2D_versatile_fluo'   'Versatile (fluorescent nuclei)'
'2D_versatile_he'     'Versatile (H&E nuclei)'
'2D_paper_dsb2018'    'DSB 2018 (from StarDist 2D paper)'
'2D_demo'             None
Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


- specify the dataset path, channel, saved folder
- the data.h5 store [frame, channel, depth, width, heighth]
- obtain the raw segmentation 

In [50]:
dataset_path = Path('/work/venkatachalamlab/Hang/00Identify_neuron/datasets/sexual_dimorphism_frame/male/20220329_male_10_processed')
ch = 1 ## specify the channel to segmentation
zoom_factor = 2 ## microfluid/NeuroPAL use 1, behaving worms use 2 if shape is (23,512,512) 
seg_path = dataset_path/'seg' ## saved folder
seg_path.mkdir(parents=True, exist_ok=True)  # Create the directory if it doesn't exist
t_idx = 0 ## select which frame
gamma = None

## hey Sina, this is the raw segmentation from the trained 3D stardist

In [19]:
label_z = get_frame_segmentation(model,model_2D,t_idx, ch, zoom_factor,dataset_path,gamma)

In [79]:
np.unique(label_z).shape ## the number of found neurons

(379,)

### - save the segmented data into h5 file

In [95]:
# save_var_h5(seg_path / (str(t_idx)+'.h5') ,[label_z],['label'])
# print("finish the segmentation of movie in the folder path: ", folder_path)

- extract the centroid from the segmentation
- use these centroid to group the 2D segmented nuclei
- apply watershed algorithm

In [54]:
from skimage.measure import regionprops

props = regionprops(label_z)
centroids = np.array([prop.centroid for prop in props])
print(centroids) 

[[  14.          216.96842105 1016.44210526]
 [  10.29411765  191.875       492.91911765]
 [  13.44047619  425.11904762  929.45238095]
 ...
 [   9.19205298  420.39735099  474.60927152]
 [  17.07042254  147.92957746  595.85915493]
 [  14.45517241  502.58965517  743.83793103]]


In [92]:
from load_func import *
from Seg_pos import *

## specify the parameters
isotropy_scale  = (5,1,1)
normalize_lim = (3,99.5) #(1,99.5)
zoom_factor = 1

In [70]:
img_original, _ = get_volume_at_frame(dataset_path/'data.h5',t_idx)
abs_pos = np.round(centroids).astype(np.int16)
gray_volume = img_original[0,ch]
apply_NucleiSegmentation = NucleiSegmentationAnnotation_test(gray_volume, model_2D, isotropy_scale,  normalize_lim, zoom_factor)
seg = apply_NucleiSegmentation.run_segmentation(abs_pos)
print(np.unique(seg).shape)

(353,)


## Hey Sina,the 'seg' is the refined segmentation 
Happy to hear from you about any questions and let me know if any of issues

In [71]:
### load annotations 
# import pandas as pd
# with h5py.File(dataset_path / 'annotations.h5', 'r') as f:
#     data = {key: f[key][:] for key in f.keys()}
    
# df = pd.DataFrame(data)
# df = df[df['t_idx']==t_idx]
# img_shape = np.array(label_z.shape)
# pos = df[['z','y','x']].values * (img_shape-1)
# pos = np.round(pos).astype(np.uint16)